# Tutorial 8: Prediction and Model Selection

#### Lecture and Tutorial Learning Goals:

By the end of this section, students will be able to:

- Explain the difference between confidence intervals for prediction and prediction confidence intervals and what elements need to be estimated to construct these intervals.

- Write a computer script to calculate these intervals. Interpret and communicate the results from that computer script.

- Give an example of a question that can be answered by predictive modelling.

- Explain the algorithms for the following variable selection methods: • Forward selection • Backward selection

- Explain when a linear regression is an appropriate model to predict new outcomes based on new values of the input variables.

- List model metrics that are suitable for evaluation of a statistical model developed for the purpose of predictive modelling (e.g., RMSE), as well as how they are calculated.

- Discuss how different estimation methods can result in different predictions.

In [ ]:
# Run this cell before continuing.
library(tidyverse)
library(broom)
library(repr)
library(digest)
library(infer)
library(gridExtra)
library(faraway)
library(mltools)
library(leaps)
library(glmnet)
library(cowplot)
source("tests_tutorial_08.R")

## 1. Prediction CI *versus* CI for Prediction

In previous lectures we have learned how to estimate LR models and used them to make inference about the population parameters. In this lecture we will learn different concepts related to *prediction*.

> **Heads up**: It is important to distinguished between *in-sample* prediction from *out-of-sample* prediction

We have seen different measures to compare the *in-sample* values of the response with their corresponding predicted values using a LR to evaluate the goodness of the model.

In this first section we are going to recognize and measure the *uncertainty* of these predictions.

Let us start by loading the dataset to be used throughout this tutorial. We will use the dataset `fat` from the library `faraway`. You can find detailed information about it in [Johnson (1996)](https://www.tandfonline.com/doi/full/10.1080/10691898.1996.11910505). This dataset contains the percentage of body fat and a whole variety of body measurements (continuous variables) of 252 men. We will use the variable `brozek` as the response variable and a subset 14 variables to build different models. 

Run the code below to create the working data frame called `fat_sample`.

In [ ]:
fat_sample <- fat %>%
  select(
    brozek, age, weight, height, adipos, neck, chest, abdom,
    hip, thigh, knee, ankle, biceps, forearm, wrist
  )

head(fat_sample,3)

The response variable `brozek` is the percent of body fat using Brozek's equation:

$$\texttt{brozek} = \frac{457}{\texttt{density}} - 414.2,$$

where body `density` is measured in $\text{g}/\text{cm}^3$.

The 14 input variables are:

- `age`: Age in $\text{years}$.
- `weight`: Weight in $\text{lb}$.
- `height`: Height in $\text{in}$.
- `adipos`: Adiposity index in $\text{kg}/\text{m}^2$.

$$\texttt{adipos} = \frac{\texttt{weight}}{\texttt{height}^2}$$

- `neck`: Neck circumference in $\text{cm}$.
- `chest`: Chest circumference in $\text{cm}$.
- `abdom`: Abdomen circumference at the umbilicus and level with the iliac crest in $\text{cm}$.
- `hip`: Hip circumference in $\text{cm}$.
- `thigh`: Thigh circumference in $\text{cm}$.
- `knee`: Knee circumference in $\text{cm}$.
- `ankle`: Ankle circumference in $\text{cm}$.
- `biceps`: Extended biceps circumference in $\text{cm}$.
- `forearm`: Forearm circumference in $\text{cm}$.
- `wrist`: Wrist circumference distal to the styloid processes in $\text{cm}$.

**Question 1.0**
<br>{points: 1}

Let's start by building a SLR using only `weight` to predict `brozek`.

Use the `lm()` function to estimate the SLR. Store this estimated model in the variable `SLR_fat`.

*Fill out those parts indicated with ..., uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# SLR_fat <- ...(..., ...)
# SLR_fat

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.0()

**Question 1.1**
<br>{points: 1}

In previous lectures, we have learned how to obtain and interpret confidence intervals for the regression parameters. 

Since the predictions are functions of the estimated LR, they also depend on the sample used! A different sample would have resulted in a different estimated LR and different predictions! As dicussed for the estimation of the regression parameters, we can obtain confidence intervals that take into account the sample-to-sample variation of the predictions as well! 

There are 2 type of intervals we can construct depending on the quantity we want to predict: *confidence intervals for prediction* and *prediction confidence intervals*

> **Heads up**: Isn't this confusing?? 

Let's start by computing *confidence intervals for prediction*. These are intervals to predict the *average* brozek index for men of different weights. 

Using `SLR_fat` and `predict`, obtain the asymptotic 95% CIP (confidence intervals for prediction). Create a dataframe, called `fat_cip`, that contains the response, the input, the predictions, and the lower and upper bounds of the intervals for each observation **in that order from left-to-right**. 

*Fill out those parts indicated with ..., uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_cip <- fat_sample  %>% 
#    select(..., ...) %>% 
#    cbind(predict(...,interval="confidence",se.fit=TRUE)$fit)  %>% 
#    mutate_if(is.numeric, round, 3)
# head(fat_cip)


# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.1()

**Question 1.2**
<br>{points: 1}

We have just calculated the 95% confidence interval for the mean brozek index for men of different weights in our sample. 

Provide a brief interpretation for the 95% confidence interval for prediction you have calculated in row 1.

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

**Question 1.3**
<br>{points: 1}

Let's now compute and interpret *prediction confidence intervals*. These are intervals to predict the (actual) brozek index for men of different weights.  

You can use `SLR_fat` and `predict` again to obtain the asymptotic 95% PI (prediction intervals) changing the argument `interval`. Create a dataframe, called `fat_pi`, that contains the response, the input, the predictions, and the lower and upper bounds of the intervals for each observation, **in that order from left to right**.

> **Heads up**: read the warning message! since your goal is to predict an actual value, it is important to note that this is not coming from a test set.

*Fill out those parts indicated with ..., uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_pi <- fat_sample  %>% 
#    select(..., ...) %>% 
#    cbind(predict(...,interval="prediction",se.fit=TRUE)$fit)  %>% 
#    mutate_if(is.numeric, round, 3)
# head(fat_pi)


# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_1.3()

**Question 1.4**
<br>{points: 1}

We have just calculated the 95% prediction interval for the brozek index of men of different weights in our sample. 

Provide a brief interpretation for the 95% prediction interval you have calculated in row 1.
Your interpretation goes here.

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

**Question 1.5**
<br>{points: 1}

Compare the confidence intervals computed in **Question 1.1** with those computed in **Question 1.3** (by row). Which confidence intervals are wider?? Respond and explain why in one or two sentences.

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## 2. Model selection

In this section you will use the **backward selection** algorithm to construct a generative model and the **forward selection** algorithm to construct a predictive model. 

> Note that the choice of which algorithm to use in each case was arbitrary and you can play with these algorithms and try other choices!

### 2.1 Selecting a generative model

In this section you will use the **backward selection** algorithm to construct a *generative model*.

Generative models are built and trained to examine the association between the response and the input variables. 

Since the same data can not be used to select and to make inference, we need 2 different datasets: a *selection* set and a *training* set. If two independent datasets are not available to select and build a generative model, we can use the data in hand and split it to create these datasets. 

> using the same data to select and estimate violates the assumptions of the analysis and invalidate results. This problem is known as a *post-inference* problem and will be further discuss in future lectures. 

**Question 2.0**
<br>{points: 1}

Let's start by randomly splitting `fat_sample` in two sets on a 70-30% basis: `training_fat` (70% of the data) and `second_set_fat` (the remaining 30%) to be used either to *select* a generative model or to *test* a predictive model.

Follow the next 3 steps to complete the code below:

1. Create an `ID` column in `fat_sample` (i.e., `fat_sample$ID`) with the row number corresponding to each man in the sample.

2. Use the function `sample_n()` to create `training_fat` (sampling *without* replacement) with 70\% of the observations coming from `fat_sample`.

3. Use `anti_join()` with `fat_sample` and `training_fat` to create `second_set_fat` by column `ID`.

4. Remove the variable `ID` used to split the data

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
set.seed(123) # DO NOT CHANGE!

# fat_sample$ID <- rownames(fat_sample)
# training_fat <- ...(..., size = nrow(fat_sample) * 0.70,
#   replace = ...
# )

# second_set_fat <- anti_join(...,
#   ...,
#   by = ...
# )

#training_fat <- training_fat[,-"ID"]
#second_set_fat <- second_set_fat[,-"ID"]

# head(training_fat)
# nrow(training_fat)

# head(second_set_fat)
# nrow(second_set_fat)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.0()

### Selecting a generative model

Although many potential input variables are available in the dataset, not all may be relevant to explain the variation of the response variable. The *subset* algorithms learned in the lecture can be used to select a subset of variable to build generative models. 

In the following questions, we will use the *backward* selection algorithm and the *adjusted* $R^2$ to select a smaller model.

> Note that we can't use the same data to select and make inference. Thus, the selection will be done using the `second_set_fat` dataset and the model will be built using teh `training_fat` data.

**Question 2.1**
<br>{points: 1}

Using only the extra data in `second_set_fat`, select a reduced LR using the **backward selection** algorithm. Recall that this method is implemented in the function `regsubsets()` from library `leaps`.

The function `regsubsets()` identifies various subsets of input variables selected for models of different sizes. The argument `x` of `regsubsets()` is analogous to `formula` in `lm()`. 

Create one object using `regsubsets()`with `second_set_fat` and call it `fat_backward_sel`. We will use `fat_bwd_summary_df` to check the results.

**Maintain any ordering of columns seen in `second_set_fat`**

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_backward_sel <- ...(
#   x=..., 
#   nvmax=...,
#   data=...,
#   method=...,
# )
# fat_backward_sel

#fat_bwd_summary <- summary(fat_backward_sel)

#fat_bwd_summary_df <- data.frame(
#    n_input_variables = 1:14,
#    RSQ = fat_bwd_summary$rsq,
#    RSS = fat_bwd_summary$rss,
#    ADJ.R2 = fat_bwd_summary$adjr2,
#)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.1()

**Question 2.2**
<br>{points: 1}

The *backward* subset algorithm selected the best model of each size. Results of the 14 models selected are stored in `fat_bwd_summary`. 

Use the *adjusted* $R^2$ of these 14 models, stored in `fat_bwd_summary_df`, to select the best generative model and indicate which input variables are in the selected model.

**A.** `age`.

**B.** `weight`.

**C.** `height`.

**D.** `adipos`.

**E.**  `neck`.

**F.**  `chest`.

**G.**  `abdom`.

**H.**  `hip`.

**I.**  `thigh`.

**J.**  `knee`.

**K.**  `ankle`.

**L.**  `biceps`.

**M.**  `forearm`.

**N.**  `wrist`.

*Assign your answers to the object `answer2.5`. Your answers have to be included in a single string indicating the correct options **in alphabetical order** and surrounded by quotes.*

In [ ]:
#Run this cell before continuing to examine the results

fat_bwd_summary_df

fat_bwd_summary 

In [ ]:
# answer2.2 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.2()

**Question 2.3**
<br>{points: 1}

Now that you have selected a subset of input variables, use the independent dataset `training_fat` to build and evaluate a *generative* model. 

Use `lm` to fit the selected model using `training_fat`, and store the results in an object called `fat_bwd_generative`. 

> Enter the selected variables in the **same order** as they are in `training_fat`. This is not statistically needed, it's only needed to autograde this question.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_bwd_generative <- ...(...,
#   ...
# )

# tidy(fat_bwd_generative)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.3()

**Question 2.4**
<br>{points: 1}

Compute the coefficient of determination $R^2$ to evaluate the goodness of fit of the model.

> Note that the evaluation is also based on data from `training_fat`

*Assign your answer to the object `answer2.4`. Your answer is a numeric object* 

In [ ]:
# *Your code goes here.*

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_2.4()

**Question 2.5**
<br>{points: 1}

Interpret the coefficient of determination $R^2$ computed in **Question 2.4** and comment on the goodness of fit of the selected model.

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

**Question 2.6**
<br>{points: 1}

Previous research has shown that while weight can be highly variable during the day and even across days, body circumference measurements (e.g., abdominal circumference) are more stable and better predictors of body fat. Using the results from **Question 2.3**, corroborate if the abdominal circumference, `abdom` is statistically associated with the percent of body fat measured by `brozek`. 

In your answer, include an interpretation of the estimated coefficients as well as the results of the *t*-tests reported using `tidy()`.

> *Your answer goes here.*

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## 3. Predictive Modelling using Linear Regression

In this section you will use the **forward** selection algorithm to select a *predictive model*.

Predictive models are built and trained to predict *new* observations. Thus, we need two types of datasets: a *training* set and a *test* set. If two independent datasets are not available to build a predictive model, we can:


- approximate the *test* MSE

or 

- use the data in hand and split it to create these datasets.


In the following exercises you will build a predictive model using the first approach.

**Question 3.0**
<br>{points: 1}

Let's start by building a predictive additive LR with *all* **14** inputs. Call this object `fat_full_OLS_add`. 

Estimate an additive LR with *all* **14** inputs against the response variable `brozek`  using `lm()` and data from `training_fat`. 

> **Order of these variables should match the column order result from `training_fat`**.

This will be our baseline model.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_full_OLS_add <- lm(...,
#   ...
# )
# fat_full_OLS_add

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.0()

**Question 3.1**
<br>{points: 1}

Using `predict()` and `fat_full_OLS_add`, obtain the (out-of-sample) predicted brozek values for men in `second_set_fat`. 

> `second_set_fat` will be used as independent *test data*

Store them in a variable called `fat_test_pred_full_OLS_add`. 

> **Keep the order of columns seen in `second_set_fat`**

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_test_pred_full_OLS_add <- ...(..., newdata = ...)
# head(fat_test_pred_full_OLS_add)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.1()

**Question 3.2**
<br>{points: 1}

We will now compute the **Mean Squared Error (MSE)** using data from the test set to evaluate the predictive model (the smaller the metric, the better the model):

> **Heads up:** a related measure commonly used is the **Root Mean Squared Error (RMSE) = $\sqrt{\text{MSE}}$**. This metric has the same units as the response.

Use the function `rmse()` from the `mltools` package to compute the $\text{RMSE}_{\text{test}}$ based on the *predicted* brozed values stored in `fat_test_pred_full_OLS_add` for men in the test set. Note that the observed brozek values for these men are in `fat_test$brozek`. 

Store this metric in a tibble called `fat_R_MSE_models` with two columns:

- `Model`: The regression model from which we will obtain the prediction accuracy.
- `R_MSE`: The $\text{RMSE}_{\text{test}}$ corresponding to the model.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_R_MSE_models <- tibble(
#   Model = "OLS Full Regression",
#   R_MSE = ...(
#     ...,
#     ...
#   )
# )
# fat_R_MSE_models

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.2()

### Selecting a predictive model

The previous model uses all input variables to predict. However, we may want to select a smaller model by using only a subset of the input variables. The *subset* algorithms that we used in previous questions can be used to build predictive models as well. 

A good predictive model would be one that minimizes the *test* MSE. However, we can not use the same set to select the model and evaluate its performance. 

Metrics such as $C_p$, AIC and BIC are computed with the *training* set and can be used to *approximate* the *test* MSE, without looking at the *test* data. 

The test set will then be used *only* to assess the predictive performance of the selected model.

**Question 3.3**
<br>{points: 1}

Using only the training data in `training_fat`, select a reduced LR using the **forward selection** algorithm. Recall that this method is implemented in the function `regsubsets()` from library `leaps`.

The function `regsubsets()` identifies various subsets of input variables selected for models of different sizes. The argument `x` of `regsubsets()` is analogous to `formula` in `lm()`. 

Create one object using `regsubsets()`with `training_fat` and call it `fat_forward_sel`. We will use `fat_fwd_summary_df` to check your results.

> **Maintain the order of columns seen in `training_fat`**

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_forward_sel <- ...(
#   ..., ...,
#   ...,
#   ...,
# )
# fat_forward_sel

#fat_fwd_summary <- summary(fat_forward_sel)

#fat_fwd_summary_df <- tibble(
#    n_input_variables = 1:14,
#    RSS = fat_fwd_summary$rss,
#    BIC = fat_fwd_summary$bic,
#    Cp = fat_fwd_summary$bic
#)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.3()

**Question 3.4**
<br>{points: 1}

Out of the fourteen best models selected for each size by the *forward* subset algorithm and stored in `fat_forward_sel`, we will select the best one in terms of the *out-of-sample* prediction accuracy, estimated by the Mallow's $C_p$. 

Use the $C_p$ computed for each model, stored in `fat_bwd_summary_df`, to select the best predictive model and indicate which input variables are in the selected model.

> **Heads up:** The most accurate model will have the smallest $C_p$. 


**A.** `age`.

**B.** `weight`.

**C.** `height`.

**D.** `adipos`.

**E.**  `neck`.

**F.**  `chest`.

**G.**  `abdom`.

**H.**  `hip`.

**I.**  `thigh`.

**J.**  `knee`.

**K.**  `ankle`.

**L.**  `biceps`.

**M.**  `forearm`.

**N.**  `wrist`.

*Assign your answers to the object `answer3.4`. Your answers have to be included in a single string indicating the correct options **in alphabetical order** and surrounded by quotes.*

In [ ]:
#Run this cell below before continuing.

fat_fwd_summary_df
summary(fat_forward_sel)

In [ ]:
# answer3.4 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.4()

**Question 3.5**
<br>{points: 1}

Use the variables selected by the forward subset algorithm to build a *predictive* model. 

Train the predictive model using `lm()` and the `training_fat` data. Call it `fat_red_OLS_add`. 

> In your model, place variables in the **same order** as they are in `training_fat`. This is not statistically needed, it's only needed to autograde this question.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_red_OLS_add <- ...(...,
#   ...
# )

# summary(fat_red_OLS_add)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.5()

**Question 3.6**
<br>{points: 1}

Use the trained model `fat_red_OLS_add` to predict the responses of the test set `second_set_fat`, and call the resulting object `fat_test_pred_red_OLS_add`. 

> In your model, place variables in the **same order** as they are in `training_fat`.

> **Hint:** Use function `predict()`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_test_pred_red_OLS_add <- ...(..., ...)

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.6()

**Question 3.7**
<br>{points: 1}

Use the function `rmse()` to compute the RMSE of predicted brozek values of men in the test set stored in `fat_test_pred_red_OLS_add`. Add this metric as another row in the tibble `fat_R_MSE_models` with `"OLS Reduced Regression"` in the column `Model` and the corresponding $\text{RMSE}_{\text{test}}$ in column `R_MSE`.

*Fill out those parts indicated with `...`, uncomment the corresponding code in the cell below, and run it.*

In [ ]:
# fat_R_MSE_models <- rbind(
#   fat_R_MSE_models,
#   tibble(
#     Model = ...
#     R_MSE = ...
#     )
#   )
# fat_R_MSE_models

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.7()

**Question 3.8**
<br>{points: 1}

Based on your results in `fat_R_MSE_models`, which model has the best *out-of-sample* prediction performance?

**A.** OLS Full Regression.

**B.** OLS Reduced Regression.

*Assign your answer to an object called `answer3.8`. Your answer should be one of `"A"` or `"B"` surrounded by quotes.*

In [ ]:
# answer3.8 <- 

# your code here
fail() # No Answer - remove if you provide an answer

In [ ]:
test_3.8()